In [1]:
import AdventOfCode
import AdventOfCode.Intcode

In [2]:
input <- dayString 11

In [3]:
startState = new (parse input)

In [4]:
produceOutput program input output index = let
    (program', input', output', index') = step program input output index
    in if output' /= output || index == index'
        then (program', output', index')
        else produceOutput program' input' output' index'

In [5]:
data Direction = L | U | R | D deriving (Show, Eq, Ord, Enum)

In [6]:
turnLeft :: Direction -> Direction
turnLeft d = toEnum $ (fromEnum d-1) `mod` 4

turnRight :: Direction -> Direction
turnRight d = toEnum $ (fromEnum d + 1) `mod` 4

In [7]:
import qualified Data.Map.Strict as Map

In [8]:
import Control.Monad.State.Strict

In [9]:
forward :: (Int, Int) -> Direction -> (Int, Int)
forward (x,y) direction = case direction of
    L -> (x-1,y)
    U -> (x,y+1)
    R -> (x+1,y)
    D -> (x,y-1)

In [10]:
data Colour = Black | White deriving (Eq, Show, Ord, Enum)

move coord direction visited program index = let
    input = Map.findWithDefault Black coord visited
    (program', output', index') = produceOutput program [fromEnum input] [] index
    in if null output'
        then visited
        else let
            [colour] = output'
            visited' = Map.insert coord (toEnum colour :: Colour) visited
            (program'', [turn], index'') = produceOutput program' [] [] index'
            direction' = case turn of
                0 -> turnLeft direction
                1 -> turnRight direction
            in move (forward coord direction') direction' visited' program'' index''

In [11]:
visited = move (0,0) U Map.empty startState 0

In [12]:
Map.size visited

1930

In [13]:
visited' = move (0,0) U (Map.fromList [((0,0), White)]) startState 0

In [14]:
visitedList = Map.toList visited'

In [15]:
ys = [l | i <- [-5,-4,-3,-2,-1,0], let l = filter (\((x,y),c) -> y == i) visitedList]

In [16]:
withIndices = map (map (\((x,_),c) -> (x,c))) ys

In [17]:
import qualified Data.Vector as V

In [18]:
allBlack = V.replicate 43 Black

In [19]:
filled = map (allBlack V.//) withIndices

In [20]:
rendered = flip map filled $ \row -> flip V.map row $ \c -> case c of
    Black -> ' '
    White -> '#'

mapM_ print (reverse rendered)

Line 1: Use lambda-case
Found:
\ c ->
  case c of
      Black -> ' '
      White -> '#'
Why not:
\case
    Black -> ' '
    White -> '#'

" ###  #### #  # #  # ####  ##  #### #  #   "
" #  # #    # #  #  # #    #  #    # #  #   "
" #  # ###  ##   #### ###  #      #  #  #   "
" ###  #    # #  #  # #    #     #   #  #   "
" #    #    # #  #  # #    #  # #    #  #   "
" #    #    #  # #  # ####  ##  ####  ##    "